In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Читаем данные

In [ ]:
temp_data = pd.read_csv('../input/daily-temperature-of-major-cities/city_temperature.csv')

In [ ]:
temp_data

# Выбираем Москву

In [ ]:
temp_data_Moscow = temp_data.loc[(temp_data.Country == 'Russia') & (temp_data.City == 'Moscow')]
temp_data_Moscow.head(31)

# Данные немного всратые:

* Russia, Moscow, 01.01.1995:       32.3 F = 0 C
* ...
* Russia, Moscow, 01.22.1995:   4.0 F = -15,5 ????????????????????

Это точно Россия?

# Переводим из фаренгейта в цельсия

In [ ]:
temp_data_Moscow.AvgTemperature = temp_data_Moscow.AvgTemperature.map(lambda x: round(((x - 32) * 5/9), 2))
temp_data_Moscow.head(50)

# Выбираем 1995 год, меняем индексы

In [ ]:
temp_for_1995 = temp_data_Moscow.loc[temp_data_Moscow.Year == 1995]
temp_for_1995 = temp_for_1995[0:len(temp_for_1995)].set_index(pd.Index([n for n in range(len(temp_for_1995))]))
temp_for_1995.head(33)


# Создаём список средних месячных температур

In [ ]:
avgMonthTemp = []
currentMonth = 1
currentDay = 0
monthTemp = 0



for i in range(len(temp_for_1995)):
    
    if temp_for_1995['Month'][i] != currentMonth:
        print(currentDay)
        
        avgMonthTemp.append(round(monthTemp/currentDay, 2))
        
        monthTemp = 0
        currentDay = 0
        currentMonth += 1
        
    if i == 364:
        currentDay += 1
        print(currentDay)
        avgMonthTemp.append(round(monthTemp/currentDay, 2))
        
        
    
    monthTemp += temp_for_1995['AvgTemperature'][i]
    currentDay += 1
    
    
        
        
print(avgMonthTemp)
        
    
    

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20,6))
sns.barplot(x=[n+1 for n in range(len(avgMonthTemp))], y=avgMonthTemp)



# Ну вроде всё норм, теперь что-нибудь посложнее

# Что там с глобальным потеплением

In [ ]:
temp_data_Moscow
# данные за 2020 не полные, так что исключу их
temp_data_Moscow_New = temp_data_Moscow.loc[temp_data_Moscow.Year != 2020]
temp_data_Moscow_New

In [ ]:
temp_data_Moscow_New = temp_data_Moscow_New[0:len(temp_data_Moscow_New)].set_index(pd.Index([n for n in range(len(temp_data_Moscow_New))]))
temp_data_Moscow_New


In [ ]:
averageYearTemperature = []

for j in range(25):
    
    avgMonthTemp = []
    currentMonth = 1
    currentDay = 0
    monthTemp = 0
    
    yearData = temp_data_Moscow_New.loc[temp_data_Moscow_New.Year == 1995+j]
    yearData = yearData[0:len(yearData)].set_index(pd.Index([n for n in range(len(yearData))]))
    print(yearData.head(1))
    
    
    for i in range(len(yearData)):
        
        if yearData['Month'][i] != currentMonth:
            #print(currentDay)
            avgMonthTemp.append(round(monthTemp/currentDay, 2))
            
            monthTemp = 0
            currentDay = 0
            currentMonth += 1
            
        if i == 364:
            currentDay += 1
            #print(currentDay)
            avgMonthTemp.append(round(monthTemp/currentDay, 2))
            
        monthTemp += yearData['AvgTemperature'][i]
        currentDay += 1

    averageYearTemperature.append(round(sum(avgMonthTemp)/12, 3))
    
    

print(averageYearTemperature)
    

# Данные говно полное, какой +1 в январе

> Europe  Russia   NaN  Moscow      1    1  2017             1.0


In [ ]:
plt.figure(figsize=(20,6))
sns.barplot(x=[n for n in range(1995, 2020)], y=averageYearTemperature)

In [ ]:
plt.figure(figsize=(20,6))
sns.regplot(x=[n for n in range(1995, 2020)], y=averageYearTemperature)

# Короче да, глобальное потепление, мы всё умрем